In [ ]:
import torch
import torch.optim as optim
import torchvision.transforms as trans
import torch.nn.functional as F

import matplotlib.pyplot as plt

from dqn_model import CNN
from replay_memory import ReplayMemory, Transition

In [ ]:
env = gym.make("CarRacing-v2")

for i_episode in range(NUM_EPISODES):
    episode_losses = []
    total_reward = 0
    RANDOM_COUNTER = 0
    NN_COUNTER = 0
    score_sliding = 0

    # Initialize the environment and state
    print("Starting episode", i_episode)
    env.reset()
    wait_for_zoom()
    state, _, _ = get_res_state([0, 0, 0])

    for t in range(10000):
        # select current action
        action_index = select_action(state)
        action = ACTIONSPACE[action_index]

        # get normalized state from current action
        next_state, reward, done = get_res_state(action)

        total_reward += reward

        reward = float(reward)
        reward = torch.tensor([reward], device=DEVICE)

        if done:
            break

        # Store the transition in memory
        MEMORY.push(state, action_index, reward, next_state, done)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        episode_loss = optimize_model()

        if episode_loss is not None:
            episode_losses.append(episode_loss)

    if len(episode_losses) > 0:
        avgloss = sum(episode_losses)/len(episode_losses)

    else:
        avgloss = 0

    TOTAL_AVG_LOSSES.append(avgloss)

    print(f"Random steps this episode {RANDOM_COUNTER}")
    print(f"NN steps this episode {NN_COUNTER}")
    print(f"Total reward in episode {i_episode} is {total_reward} and awg loss is {avgloss}")
    TOTAL_SCORES.append(total_reward)
    score_sliding = sum(TOTAL_SCORES) / len(TOTAL_SCORES)
    SCORE_AVG.append(score_sliding)

# show graphs
plt.plot(TOTAL_SCORES)
plt.plot(SCORE_AVG)
plt.title(f"Scores per episode, batch size {BATCH_SIZE}")
plt.show()
plt.plot(TOTAL_AVG_LOSSES)
plt.title(f"Avg losses per episode")
plt.show()

print('Complete')

env.close()
